# Initialization

Test notebook for the C-MAPPS benchmark. Approach using MLP. 

First we import the necessary packages and create the global variables.

In [8]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.font_manager
import matplotlib
import seaborn as sns
import pandas as pd
import logging
import random
#import plottingTools
from datetime import datetime
from sklearn.covariance import EllipticEnvelope
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neural_network import MLPClassifier
from mpl_toolkits.mplot3d import Axes3D
#from dataManagement import DataManagerDamadics

from IPython.display import display, HTML
%matplotlib notebook

global constRUL

constRUL = 125
time_window = 30
rul_vector = None

In [9]:
def compute_training_RUL(df_row):
    
    global constRUL
    
    if rul_vector[int(df_row['Unit Number']) - 1] - df_row['Cycle'] > constRUL:
        return constRUL
    else:
        return rul_vector[int(df_row['Unit Number']) - 1] - df_row['Cycle']

In [10]:
def get_X_y_from_df(df, time_window, features):
    
    n_m = df.shape[0]
    n_x = len(features)
    
    df_values = df[features].values
    targets = df['RUL'].values
    
    X, y = np.empty([n_m-time_window+1, n_x*time_window]), np.empty([n_m-time_window+1, 1])
    
    for i in range(n_m-time_window+1):
        
        time_window_samples = df_values[i:i+time_window]
        X[i,:] = np.squeeze(time_window_samples.reshape(1,-1))
        y[i] = targets[i+time_window-1]
        
        #Do not use this implementation, its slow everytime a new dataframe is created
        '''temp_df = df.iloc[i:i+time_window][features]
        temp_row = np.squeeze(temp_df.values.reshape(1,-1))
        X[i,:] = temp_row
        y[i] = df.iloc[i+time_window-1]['RUL']'''
    
    return X, y

# Retrieve and Reshape data

Get the data from the text files, store it in a Pandas Dataframe and reshape it as appropiately.

In [11]:
def retrieve_and_reshape_data(from_file, columns, time_window):
    '''
    T2        - Total temperature at fan inlet      R
    T24       - Total temperature at lpc outlet     R
    T30       - Total temperature at hpc outlet     R
    T50       - Total temperature at LPT outlet     R
    P2        - Pressure at fan inlet               psia
    P15       - Total pressure in bypass-duct       psia
    P30       - Total pressure at HPC outlet        psia
    Nf        - Physical fan speed                  rpm
    Nc        - Physical core speed                 rpm
    epr       - Engine Pressure ratio (P50/P2)      --
    Ps30      - Static Pressure at HPC outlet       psia
    phi       - Ratio fuel flow to Ps30             pps/psi
    NRf       - corrected fan speed                 rpm
    NRc       - Corrected core speed                rpm
    BPR       - Bypass ratio                        --
    farB      - Burner fuel-air ratio               --
    htBleed   - Bleed enthalpy                      --
    Nf_dmd    - Demanded fan speed                  rpm
    PCNfR_dmd - Demanded corrected fan speed        rpm
    W31       - HPT coolant bleed                   lbm/s
    W32       - LPT coolant bleed                   lbm/s
    '''

    
    df = pd.read_csv(from_file ,sep='\s+',header=None)

    col_names = {0:'Unit Number', 1:'Cycle', 2:'Op. Settings 1', 3:'Op. Settings 2', 4:'Op. Settings 3', 5:'T2',
                6:'T24', 7:'T30', 8:'T50', 9:'P2', 10:'P15', 11:'P30', 12:'Nf', 13:'Nc', 14:'epr', 15:'Ps30', 
                16:'phi', 17:'NRf', 18:'NRc', 19:'BPR', 20:'farB', 21:'htBleed', 22:'Nf_dmd', 23:'PCNfR_dmd', 
                24:'W31', 25:'W32'}

    df.rename(columns=col_names, inplace=True)

    gruoped_by_unit = df.groupby('Unit Number')
    rul_vector = gruoped_by_unit.size().values

    df['RUL'] = df.apply(compute_training_RUL, axis = 1)
    selected_features_rul = columns[:]
    selected_features_rul.append('RUL')
    df_selected_features = df[selected_features_rul]

    X, y = get_X_y_from_df(df_selected_features, time_window, selected_features)
    
    return X, y

    '''display(df_selected_features)
    print(X.shape)
    print(X)
    print(y.shape)
    print(y)'''

In [12]:

data_file = '../CMAPSSData/train_FD001.txt'

#Selected as per CNN paper
selected_features = ['T2', 'T24', 'T30', 'P15', 'P30', 'Nf', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'farB', 'PCNfR_dmd', 'W31']

#Get the X and y matrices with the specified time window
X, y = retrieve_and_reshape_data(data_file, selected_features, time_window)

print(X.shape)
print(X)
print(y.shape)
print(y)





TypeError: ("'NoneType' object is not subscriptable", 'occurred at index 0')